# Ejercicio: Escalado de características y tasa de aprendizaje (Multi-variable)

## Objetivos
En este ejercicio usted:
- Utilizará las rutinas de múltiples variables desarrolladas en el ejercicio anterior
- Ejecutará descenso de gradiente en un conjunto de datos con múltiples características
- Explorará el impacto de la *tasa de aprendizaje alpha* en el descenso de gradiente
- Mejorará el rendimiento del descenso de gradiente mediante *escalado de características* usando normalización z-score

## Herramientas
Utilizará las funciones desarrolladas en el ejercicio anterior así como matplotlib y NumPy. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ejerc_utils_multi import  load_house_data, run_gradient_descent 
from ejerc_utils_multi import  norm_plot, plt_equal_scale, plot_cost_i_w
from ejerc_utils_common import dlc
np.set_printoptions(precision=2)
plt.style.use('./deeplearning.mplstyle')

## Notación

| Notación General | Descripción | Python (si aplica) |
|:----------------|:------------------------------------------------------------|:---------------------|
| $a$ | escalar, no en negrita | |
| $\mathbf{a}$ | vector, en negrita | |
| $\mathbf{A}$ | matriz, mayúscula en negrita | |
| **Regresión** | | |
| $\mathbf{X}$ | matriz de ejemplos de entrenamiento | `X_train` |
| $\mathbf{y}$ | objetivos de ejemplos de entrenamiento | `y_train` |
| $\mathbf{x}^{(i)}$, $y^{(i)}$ | $i$-ésimo ejemplo de entrenamiento | `X[i]`, `y[i]` |
| $m$ | número de ejemplos de entrenamiento | `m` |
| $n$ | número de características en cada ejemplo | `n` |
| $\mathbf{w}$ | parámetro: peso | `w` |
| $b$ | parámetro: sesgo | `b` |
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | Resultado de la evaluación del modelo en $\mathbf{x}^{(i)}$ parametrizado por $\mathbf{w},b$: $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$ | `f_wb` |
|$\frac{\partial J(\mathbf{w},b)}{\partial w_j}$| el gradiente o derivada parcial del costo respecto a un parámetro $w_j$ |`dj_dw[j]`| 
|$\frac{\partial J(\mathbf{w},b)}{\partial b}$| el gradiente o derivada parcial del costo respecto a un parámetro $b$| `dj_db`|

# Enunciado del Problema

Como en los ejercicios anteriores, usará el ejemplo motivador de predicción de precios de viviendas. El conjunto de datos de entrenamiento contiene muchos ejemplos con 4 características (tamaño, habitaciones, pisos y antigüedad) mostrados en la tabla a continuación. Nota: en este ejercicio, la característica Tamaño está en metros² mientras que en ejercicios anteriores se utilizó miles de metros². Este conjunto de datos es más grande que el del ejercicio anterior.

Queremos construir un modelo de regresión lineal usando estos valores para luego predecir el precio de otras casas, por ejemplo, una casa de 1200 metros², 3 habitaciones, 1 piso, 40 años de antigüedad.

## Conjunto de datos: 
| Tamaño (metros²) | Número de Habitaciones | Número de Pisos | Antigüedad de la Casa | Precio (miles de dólares) |
|----------------|----------------------|-----------------|----------------------|---------------------------|
| 952            | 2                    | 1               | 65                   | 271.5                     |
| 1244           | 3                    | 2               | 64                   | 232                       |
| 1947           | 3                    | 2               | 17                   | 509.8                     |
| ...            | ...                  | ...             | ...                  | ...                       |


In [ ]:
# cargar el conjunto de datos
X_train, y_train = load_house_data()
X_features = ['tamaño(metros²)','bedrooms','floors','age']

Visualicemos el conjunto de datos y sus características graficando cada característica versus el precio.

In [ ]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("Precio (miles)")
plt.show()

Graficar cada característica vs. el objetivo, precio, da una idea de qué características tienen mayor influencia en el precio. Arriba, aumentar el tamaño también aumenta el precio. Las habitaciones y los pisos no parecen tener un gran impacto en el precio. Las casas más nuevas tienen precios más altos que las casas más antiguas.

<a name="toc_15456_5"></a>
## Descenso de Gradiente con Múltiples Variables
Aquí están las ecuaciones que desarrolló en el ejercicio anterior sobre descenso de gradiente para múltiples variables.:

$$\begin{align*} \text{repetir}&\text{ hasta convergencia:} \; \lbrace \newline\;
& w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{para j = 0..n-1}\newline
&b\ \ := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

donde, n es el número de características, los parámetros $w_j$,  $b$, se actualizan simultáneamente y donde  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3}
\end{align}
$$
* m es el número de ejemplos de entrenamiento en el conjunto de datos

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ es la predicción del modelo, mientras que $y^{(i)}$ es el valor objetivo


## Tasa de Aprendizaje

Las lecciones discutieron algunos de los problemas relacionados con el ajuste de la tasa de aprendizaje $\alpha$. La tasa de aprendizaje controla el tamaño de la actualización de los parámetros. Ver la ecuación (1) anterior. Es compartida por todos los parámetros.

Ejecutemos descenso de gradiente y probemos algunos valores de $\alpha$ en nuestro conjunto de datos.

### $\alpha$ = 9.9e-7

In [ ]:
# establecer alpha en 9.9e-7
_, _, hist = run_gradient_descent(X_train, y_train, 10, alpha = 9.9e-7)

Parece que la tasa de aprendizaje es demasiado alta. La solución no converge. El costo está *aumentando* en lugar de disminuir. Grafiquemos el resultado:

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

El gráfico de la derecha muestra el valor de uno de los parámetros, $w_0$. En cada iteración, sobrepasa el valor óptimo y como resultado, el costo termina *aumentando* en lugar de acercarse al mínimo. Tenga en cuenta que esto no es una imagen completamente precisa ya que hay 4 parámetros que se modifican en cada pasada y no solo uno. Este gráfico solo muestra $w_0$ con los otros parámetros fijos en valores benignos. En este y en gráficos posteriores puede notar que las líneas azul y naranja están ligeramente desfasadas.


### $\alpha$ = 9e-7
Probemos un valor un poco menor y veamos qué sucede.

In [ ]:
# establecer alpha en 9e-7
_,_,hist = run_gradient_descent(X_train, y_train, 10, alpha = 9e-7)

El costo disminuye durante toda la ejecución mostrando que alpha no es demasiado grande. 

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

A la izquierda, se observa que el costo disminuye como debe ser. A la derecha, se puede ver que $w_0$ todavía oscila alrededor del mínimo, pero el costo disminuye con cada iteración en lugar de aumentar. Note arriba que `dj_dw[0]` cambia de signo en cada iteración a medida que `w[0]` salta sobre el valor óptimo.
Este valor de alpha convergerá. Puede variar el número de iteraciones para ver cómo se comporta.

### $\alpha$ = 1e-7
Probemos un valor aún más pequeño para $\alpha$ y veamos qué sucede.

In [ ]:
# establecer alpha en 1e-7
_,_,hist = run_gradient_descent(X_train, y_train, 10, alpha = 1e-7)

El costo disminuye durante toda la ejecución mostrando que $\alpha$ no es demasiado grande. 

In [ ]:
plot_cost_i_w(X_train,y_train,hist)

A la izquierda, se observa que el costo disminuye como debe ser. A la derecha, se puede ver que $w_0$ se acerca al mínimo sin oscilaciones. `dj_w0` es negativo durante toda la ejecución. Esta solución también convergerá.

## Escalado de Características (escalado de x/entradas)

Las lecciones describieron la importancia de reescalar el conjunto de datos para que las características tengan un rango similar.
Si le interesa el detalle de por qué esto es importante, haga clic en el encabezado 'detalles' a continuación. Si no, la siguiente sección mostrará cómo implementar el escalado de características.

<details>
<summary>
    <font size='3', color='darkgreen'><b>Detalles</b></font>
</summary>

Veamos nuevamente la situación con $\alpha$ = 9e-7. Este valor está bastante cerca del máximo que podemos establecer para $\alpha$ sin que diverja. Esta es una ejecución corta mostrando las primeras iteraciones:

<figure>
    <img src="./images/C1_W2_Lab06_ShortRun.PNG" style="width:1200px;" >
</figure>

Arriba, aunque el costo está disminuyendo, es claro que $w_0$ progresa mucho más rápido que los otros parámetros debido a su gradiente mucho mayor.

El gráfico siguiente muestra el resultado de una ejecución muy larga con $\alpha$ = 9e-7. Esto toma varias horas.

<figure>
    <img src="./images/C1_W2_Lab06_LongRun.PNG" style="width:1200px;" >
</figure>
    
Arriba, puede ver que el costo disminuyó lentamente después de la reducción inicial. Note la diferencia entre `w0` y `w1`,`w2`,`w3` así como entre `dj_dw0` y `dj_dw1-3`. `w0` alcanza su valor casi final muy rápido y `dj_dw0` disminuye rápidamente a un valor pequeño mostrando que `w0` está cerca del valor final. Los otros parámetros se redujeron mucho más lentamente.

¿Por qué ocurre esto? ¿Hay algo que podamos mejorar? Vea abajo:
<figure>
    <center> <img src="./images/C1_W2_Lab06_scale.PNG"   ></center>
</figure>   

La figura anterior muestra por qué los $w$ se actualizan de manera desigual. 
- $\alpha$ es compartido por todas las actualizaciones de parámetros ($w$ y $b$).
- el término de error común se multiplica por las características para los $w$ (no para $b$).
- las características varían significativamente en magnitud haciendo que algunas se actualicen mucho más rápido que otras. En este caso, $w_0$ se multiplica por 'size(sqft)', que generalmente es > 1000, mientras que $w_1$ se multiplica por 'number of bedrooms', que generalmente es 2-4. 
    
La solución es el escalado de características.

Las lecciones discutieron tres técnicas diferentes: 
- Escalado de características, que esencialmente divide cada característica positiva por su valor máximo, o más generalmente, reescala cada característica por su mínimo y máximo usando (x-min)/(max-min). Ambas formas normalizan las características al rango de -1 a 1, donde el primer método funciona para características positivas y es simple, y el segundo funciona para cualquier característica.
- Normalización por la media: $x_i := \dfrac{x_i - \mu_i}{max - min} $ 
- Normalización z-score, que exploraremos a continuación. 


### Normalización z-score 
Después de la normalización z-score, todas las características tendrán media 0 y desviación estándar 1.

Para implementar la normalización z-score, ajuste sus valores de entrada como se muestra en esta fórmula:
$$x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} \tag{4}$$ 
donde $j$ selecciona una característica o columna en la matriz $\mathbf{X}$. $µ_j$ es la media de todos los valores para la característica (j) y $\sigma_j$ es la desviación estándar de la característica (j).
$$
\begin{align}
\mu_j &= \frac{1}{m} \sum_{i=0}^{m-1} x^{(i)}_j \tag{5}\\
\sigma^2_j &= \frac{1}{m} \sum_{i=0}^{m-1} (x^{(i)}_j - \mu_j)^2  \tag{6}
\end{align}
$$

**Nota de implementación:** Al normalizar las características, es importante guardar los valores usados para la normalización: la media y la desviación estándar utilizadas en los cálculos. Después de aprender los parámetros del modelo, a menudo queremos predecir precios de casas que no hemos visto antes. Dado un nuevo valor x (área de la sala y número de habitaciones), primero debemos normalizar x usando la media y desviación estándar previamente calculadas del conjunto de entrenamiento.

**Implementación**

In [ ]:
def zscore_normalize_features(X):
    """
    calcula X, normalizado z-score por columna
    
    Argumentos:
      X (ndarray (m,n))     : datos de entrada, m ejemplos, n características
      
    Retorna:
      X_norm (ndarray (m,n)): entrada normalizada por columna
      mu (ndarray (n,))     : media de cada característica
      sigma (ndarray (n,))  : desviación estándar de cada característica
    """
    # encontrar la media de cada columna/característica
    mu     = np.mean(X, axis=0)                 # mu tendrá forma/shape (n,)
    # encontrar la desviación estándar de cada columna/característica
    sigma  = np.std(X, axis=0)                  # sigma tendrá forma/shape (n,)
    # restar la media y dividir por la desviación estándar para cada columna
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)

Veamos los pasos involucrados en la normalización z-score. El gráfico a continuación muestra la transformación paso a paso.

In [ ]:
mu     = np.mean(X_train,axis=0)   
sigma  = np.std(X_train,axis=0) 
X_mean = (X_train - mu)
X_norm = (X_train - mu)/sigma      

fig,ax=plt.subplots(1, 3, figsize=(12, 3))
ax[0].scatter(X_train[:,0], X_train[:,3])
ax[0].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[0].set_title("no-normalizado")
ax[0].axis('equal')

ax[1].scatter(X_mean[:,0], X_mean[:,3])
ax[1].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[1].set_title(r"X - $\mu$")
ax[1].axis('equal')

ax[2].scatter(X_norm[:,0], X_norm[:,3])
ax[2].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[2].set_title(r"Z-score normalizado")
ax[2].axis('equal')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle("distribución de entradas (características) antes, durante, después de la normalización")
plt.show()

El gráfico anterior muestra la relación entre dos de los parámetros del conjunto de entrenamiento, "antigüedad" y "tamaño (pies²)". *Estos se grafican con escala igual*. 
- Izquierda: Sin normalizar: El rango de valores o la varianza de la característica 'tamaño (pies²)' es mucho mayor que la de antigüedad.
- Centro: El primer paso elimina la media o valor promedio de cada característica. Esto deja las características centradas en cero. Es difícil ver la diferencia para la característica 'antigüedad', pero 'tamaño (pies²)' está claramente alrededor de cero.
- Derecha: El segundo paso divide por la desviación estándar. Esto deja ambas características centradas en cero y con una escala similar.

Normalicemos los datos y compáremoslos con los datos originales.

In [ ]:
# normalizar las entradas (características) originales
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Rango pico a pico por columna en X original: {np.ptp(X_train,axis=0)}")   
print(f"Rango pico a pico por columna en X normalizado: {np.ptp(X_norm,axis=0)}")

El rango pico a pico de cada columna se reduce de un factor de miles a un factor de 2-3 mediante la normalización.

In [ ]:
fig,ax=plt.subplots(1, 4, figsize=(12, 3))
for i in range(len(ax)):
    norm_plot(ax[i],X_train[:,i],)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("conteo");
fig.suptitle("distribución de entradas (características) antes de la normalización")
plt.show()
fig,ax=plt.subplots(1,4,figsize=(12,3))
for i in range(len(ax)):
    norm_plot(ax[i],X_norm[:,i],)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("conteo"); 
fig.suptitle("distribución de entradas (características) después de la normalización")

plt.show()

Observe arriba que el rango de los datos normalizados (eje x) está centrado alrededor de cero y aproximadamente entre +/- 2. Lo más importante es que el rango es similar para cada característica.

Volvamos a ejecutar nuestro algoritmo de descenso de gradiente con los datos normalizados.
Observe el **valor mucho mayor de alpha**. Esto acelerará el descenso de gradiente.

In [ ]:
w_norm, b_norm, hist = run_gradient_descent(X_norm, y_train, 1000, 1.0e-1, )

¡Las características escaladas logran resultados muy precisos **mucho, mucho más rápido**! Observe que el gradiente de cada parámetro es diminuto al final de esta ejecución relativamente corta. Una tasa de aprendizaje de 0.1 es un buen inicio para regresión con características normalizadas.
Grafiquemos nuestras predicciones versus los valores objetivo. Nota: la predicción se hace usando la característica normalizada mientras que el gráfico se muestra usando los valores originales.

In [ ]:
# Predice el objetivo usando las entradas (características) X normalizadas
m = X_norm.shape[0]
yp = np.zeros(m)
for i in range(m):
    yp[i] = np.dot(X_norm[i], w_norm) + b_norm

    # Grafica las predicciones y objetivos versus las entradas (características) X originales    
fig,ax=plt.subplots(1,4,figsize=(12, 3),sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train, label = 'objetivo')
    ax[i].set_xlabel(X_features[i])
    ax[i].scatter(X_train[:,i],yp,color=dlc["dlorange"], label = 'predicción')
ax[0].set_ylabel("Precio"); ax[0].legend();
fig.suptitle("Objetivo versus predicción usando el modelo normalizado z-score")
plt.show()

Los resultados se ven bien. Algunos puntos a destacar:
- con múltiples características, ya no podemos tener un solo gráfico mostrando resultados versus características.
- al generar el gráfico, se usaron las características normalizadas. Cualquier predicción usando los parámetros aprendidos de un conjunto de entrenamiento normalizado también debe ser normalizada.

**Predicción**
El objetivo de generar nuestro modelo es usarlo para predecir precios de viviendas que no están en el conjunto de datos. Predigamos el precio de una casa de 1200 pies², 3 habitaciones, 1 piso, 40 años de antigüedad. Recuerde que debe normalizar los datos con la media y desviación estándar obtenidas al normalizar los datos de entrenamiento. 

In [ ]:
# Primero, normalizar nuestro ejemplo.
x_house = np.array([1200, 3, 1, 40])
x_house_norm = (x_house - X_mu) / X_sigma
print(x_house_norm)
x_house_predict = np.dot(x_house_norm, w_norm) + b_norm
print(f" precio predicho de una casa de 1200 metros², 3 habitaciones, 1 piso, 40 años de antigüedad = ${x_house_predict*1000:0.0f}")

**Contornos de Costo**  
Otra forma de ver el escalado de características es en términos de los contornos de costo. Cuando las escalas de las características no coinciden, el gráfico de costo versus parámetros en un gráfico de contorno es asimétrico. 

En el gráfico de abajo, la escala de los parámetros está igualada. El gráfico de la izquierda es el contorno de costo de w[0], los pies cuadrados versus w[1], el número de habitaciones antes de normalizar las características. El gráfico es tan asimétrico que las curvas que completan los contornos no son visibles. En contraste, cuando las características están normalizadas, el contorno de costo es mucho más simétrico. El resultado es que las actualizaciones de los parámetros durante el descenso de gradiente pueden avanzar igual para cada parámetro. 


In [ ]:
plt_equal_scale(X_train, X_norm, y_train)


## ¡Felicitaciones!
En este ejercicio usted:
- utilizó las rutinas para regresión lineal con múltiples características desarrolladas en ejercicios anteriores
- exploró el impacto de la tasa de aprendizaje $\alpha$ en la convergencia 
- descubrió el valor del escalado de características usando normalización z-score para acelerar la convergencia

## Reconocimiento Legal
Los datos de viviendas fueron derivados del [Ames Housing dataset](http://jse.amstat.org/v19n3/decock.pdf) compilado por Dean De Cock para su uso en educación en ciencia de datos.